### Categorical Dataset. The example was designed for input data with numerical or ordinal input attributes, experiment with categorical input data and splits that may use equality instead of ranking.

W sytuacji, gdy dana cecha (feature, kolumna datasetu X) nie jest wartością ciągłą (np. cena samochodu), ale dyskretną, to znaczy pochodzącą ze skończonego zbioru, którego nie da się w sensowny sposób posortować (np. marka samochodu), nie możemy zdefiniować podziału przez threshold.

Co można zrobić w takiej sytuacji:

A) Jedna z dwóch prostych modyfikacji datasetu X (to rozwiązanie __nie__ jest dopuszczalne w Zadaniu 7.):
* Tworzymy listę wartości danej cechy, ustawiając je w losowej kolejności, np. [Syrena, Polonez, Mercedes, Wołga], a następni zastępujemy je przez kolejne liczby (w tym wypadku przez [0, 1, 2, 3]) i dalej traktujemy jako cechę ciągłą.
* Kodujemy cechy "one-hot" - każda kolumna zastępowana jest przez tyle kolumn, ile możliwych _wartości_ ma dana cecha; zawsze dokładnie jedna z nowych kolumn ma wartość 1, a pozostałe 0.

https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

B) Musimy zmienić algorytm uczenia drzewa (można wybrać jedno z poniższych jako dopuszczalne rozwiązanie). W obu poniższych przypadkach już na etapie trenowania musimy znać wszystkie możliwe wartości wszystkich cech. Jeśli w zbiorze testowym pojawi się niezaobserwowana dotychczas wartość (np. Volkswagen), to trzeba sobie z nią jakoś poradzić. Możemy np. zamienić ją na losową znaną cechę, albo przy każdym splicie zdefiniować węzeł, który domyślnie zbiera wszystkie nieznane wartości cechy.
* Jeśli dana cecha przyjmuje $k$ możliwych wartości, to splitując ją tworzymy $k$ węzłów zamiast standardowych dwóch - http://www.saedsayad.com/decision_tree.htm (tu np. cecha "Outlook" rozbija się na 3 węzły: Sunny, Overcast, Rainy); niepolecane, ponieważ przy dużej liczbie wartości danej cechy mamy overfitting (drzewo robi się bardzo płytkie).
* https://people.eecs.berkeley.edu/~russell/classes/cs194/f11/lectures/CS194%20Fall%202011%20Lecture%2008.pdf - slajd 29 oraz 30 - podobne do pierwszego rozwiązania z grupy A), ale tutaj kolejność jest dobierana w specjalny sposób; to jest lepsze rozwiązanie.

_[Jeśli wystarczy czasu, to możemy np. przeliczyć przy tablicy przykład z "Outlook" na dwa powyższe sposoby.]_

### Cross Entropy. Another cost function for evaluating splits is cross entropy (logloss). You could implement and experiment with this alternative cost function.

Jeśli etykiety danych (y) są dyskretne, to możemy użyć jednego z dwóch kryteriów oceny jakości splitu:

* Gini https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/ (to rozwiązanie __nie__ jest dopuszczalne w Zadaniu 7.),
* Cross Entropy albo Kryterium Entropijne albo Information Gain itp. - http://www.saedsayad.com/decision_tree.htm http://edu.pjwstk.edu.pl/wyklady/adn/scb/wyklad10/w10.htm

### Regression. Adapt the tree for regression using a different cost function and method for creating terminal nodes.

Jeśli etykiety danych (y) są ciągłe, to nie możemy użyć powyższych kryteriów jakości splitu. Trzeba zamiast tego skorzystać z kryterium redukcji wariancji - nasz gain to różnica wariancji zbioru przed splitem oraz sumy wariancji dwóch (lub więcej) zbiorów po dokonaniu splitu:

https://en.wikipedia.org/wiki/Decision_tree_learning#Variance_reduction

Można zapisać wzór na estymator wariancji w inny sposób:

https://en.wikipedia.org/wiki/Variance#Population_variance

A najlepiej użyć odpowiedniej funkcji z numpy:

https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.var.html

Uwaga - musimy tu użyć tzw. __biased__ estimator of variance - dlaczego nie da się użyć wersji unbiased?

### Tree Pruning. An important technique for reducing overfitting of the training dataset is to prune the trees. Investigate and implement tree pruning methods.

Dopuszczalne rozwiązanie to wybranie jednej z metod typu pre-pruning albo post-pruning:

http://www.saedsayad.com/decision_tree_overfitting.htm

https://en.wikipedia.org/wiki/Pruning_(decision_trees)

https://www.displayr.com/machine-learning-pruning-decision-trees/

Reduced Error Pruning wydaje się być najprostsze:

https://courses.cs.washington.edu/courses/cse473/98wi/slides/0311-learning/sld017.htm

W wypadku etykiet dyskretnych zastępujemy węzeł przez najczęstszą etykietę. W wypadku etykiet ciągłych zastępujemy węzeł przez średnią arytmetyczną.

Uwaga! Uczenie zarówno z pre- jak i post-pruningiem musi się rozpocząć od podzielenia zbioru treningowego `(X, y)` na zbiór treningowy i walidacyjny. W tym celu używamy metody `train_test_split`:

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

`(X_train, y_train)` - zbiór treningowy.

`(X_test, y_test)` - zbiór walidacyjny, w tym wypadku oznaczamy go jako `(X_valid, y_valid)` (zasada podziału jest taka sama; stosujemy inną nazwę, aby podkreślić, w jakim celu będziemy używać tych danych).

`test_size` można np. ustawić na stałe na 20% albo zostawić jako parametr `__init__`'a.

Uczenie drzewa odbywa się tylko na części treningowej, natomiast część testowa używana jest do pruningu. Tylko w ten sposób pruning ma sens, bo działamy na __nowych danych__, których drzewo nie widziało podczas uczenia.

Decyzję o tym, czy należy zastosować pruning na danym węźle podejmujemy na podstawie metryk, które chcemy optymalizować. W wypadku etykiet dyskretnych jest to `accuracy_score`, a w wypadku etykiet ciągłych `mean_squared_error`.

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html

`y_true` oznacza tutaj `y_valid`, natomiast `y_pred` oznacza predykcję drzewa na danych `X_valid`.

`y_pred` najlepiej uzyskać poprzez:

`y_pred = self.predict(X_valid)`

Oczywiście trzeba tę metodę wywołać dwa razy, w międzyczasie dokonując pruningu, a potem podjąć decyzję, czy chcemy go cofnąć.

### Testowanie

Aby przetestować swoje rozwiązanie można np. znaleźć jakiś ciekawy dataset i porównać się z drzewami zaimplementowanymi w `scikit-learn`:

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Dobrym źródłem jest np.

http://archive.ics.uci.edu/ml/index.php

albo

http://scikit-learn.org/stable/datasets/index.html#toy-datasets

Przesyłając rozwiązanie proszę napisać (w mailu, w kodzie w komentarzu itp.), na jakim datasecie było ono testowane (podać link), jaki wynik osiągnęły metody z `scikit-learn` oraz jaki wynik osiągnął zaimplementowany model. Testowanie jest __nieobowiązkowe__, aczkolwiek wskazane. Można też użyć zupełnie innego datasetu lub nawet stworzyć własny.

Oczywiście oceniana będzie poprawność kodu, a nie wynik.

### Przedłużony termin oddawania Zadania 7.

W związku z przedłużonym terminem osoby, które wysłały już rozwiązanie tego zadania mają __wyjątkowo__ możliwość poprawienia swojego rozwiązania (oczywiście jeśli jest taka potrzeba) tak, aby było ono zgodne z powyższymi wytycznymi.